# Alignment rates of RNAseq reads to F. heteroclitus reference 

In [297]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import jaccard distances from sourmash

In [298]:
sourmash_comp_matrix = pd.read_csv("../evaluation/all_v_ncbi.cmp.csv")

In [299]:
rows = sourmash_comp_matrix.columns

In [300]:
sourmash_comp_matrix = sourmash_comp_matrix.set_index(rows)

In [301]:
sourmash_comp_matrix['species'] =  sourmash_comp_matrix.index.values

In [302]:
species = sourmash_comp_matrix.species.str.split(".",n=1,expand=True)[0]

In [303]:
sourmash_comp_matrix = sourmash_comp_matrix.set_index(species)
sourmash_comp_matrix.index.name = 'species'

In [304]:
x_axis_jaccard = sourmash_comp_matrix[['rna.fa']]

In [305]:
x_axis_jaccard = x_axis_jaccard[:-1]

In [306]:
x_axis_jaccard[['rna.fa']] = x_axis_jaccard[['rna.fa']].astype('float')

In [307]:
x_axis_jaccard

rna.fa
species                     
A_xenica            0.065359
F_catanatus         0.109070
F_chrysotus         0.066424
F_diaphanus         0.114249
F_grandis           0.121405
F_heteroclitusMDPL  0.221701
F_heteroclitusMDPP  0.201000
F_notatus           0.073891
F_notti             0.089826
F_olivaceous        0.078874
F_parvapinis        0.050145
F_rathbuni          0.097641
F_sciadicus         0.084369
F_similis           0.065499
F_zebrinus          0.082327
L_goodei            0.054161
L_parva             0.053764

# import alignment data

In [308]:
alignment_rate_dir = "../evaluation/alignment_rates_comparison/"
alignment_rate_files = os.listdir(alignment_rate_dir)

In [309]:
alignment_rate_files

['bwa_genome_clipping_alignmentrates',
 'bwa_genome_mismatchgap_alignrates',
 'bwa_txome_clipping_alignrates',
 'STAR_genome_alignmentrates',
 'salmon_denovo_alignrates',
 'bwa_txome_mismatchgap_alignrates',
 'hisat2_genome_alignmentrate',
 'bwa_genome_alignmentrates',
 'bwa_transcriptome_alignmentrates',
 'salmon_Fhet_alignrates']

# salmon with F. het NCBI transcriptoms

In [310]:
salmon_file = alignment_rate_dir + "salmon_Fhet_alignrates"
salmon_data = pd.read_csv(salmon_file,sep=" ",header=None)
rate = salmon_data[[0,7]]
rate.columns = ['sample_info','perc_info']
rate['salmon_alignment_rate'] = rate.perc_info.str.split('%',n=1,expand=True)[0]
rate['sample'] = rate.sample_info.str.split("/",n=2,expand=True)[1]
df = rate[['sample','salmon_alignment_rate']]
df = df.drop_duplicates(subset='sample', keep='first', inplace=False)
df[['genus','species1','other1','other2']] = df['sample'].str.split('_',expand=True)
df["species"] = df["genus"] + "_" + df["species1"]
df = df[['sample','species','salmon_alignment_rate']]
df[['salmon_alignment_rate']] = df[['salmon_alignment_rate']].astype('float')

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [311]:
# get mean and sd of df_salmon groupby species
df_salmon_mean = df.groupby("species")["salmon_alignment_rate"].agg([np.mean, np.std])
df_salmon_mean.columns = ['mean_salmonFhet','std_salmonFhet']

In [312]:
# merge with jaccard distances from sourmash
df_salmon = pd.merge(df_salmon_mean,x_axis_jaccard,on = "species")

In [313]:
df_salmon

mean_salmonFhet  std_salmonFhet    rna.fa
species                                                      
A_xenica                  55.475511        1.766508  0.065359
F_catanatus               59.184429        4.343153  0.109070
F_chrysotus               54.770188        1.296955  0.066424
F_diaphanus               61.246000        2.470607  0.114249
F_grandis                 57.420867       12.006247  0.121405
F_heteroclitusMDPL        64.552211        5.273074  0.221701
F_heteroclitusMDPP        65.770489        4.722945  0.201000
F_notatus                 56.591278        2.152217  0.073891
F_notti                   55.536150        0.617092  0.089826
F_olivaceous              55.028838        3.442983  0.078874
F_parvapinis              44.248000        2.866265  0.050145
F_rathbuni                61.714956        1.894362  0.097641
F_sciadicus               56.682275        1.876854  0.084369
F_similis                 48.177767        4.917644  0.065499
F_zebrinus                52.972925        3.926850  0.082327
L_goodei                  46.687689        2.752060  0.054161
L_parva                   49.292456        4.072149  0.053764

# salmon with species-specific de novo transcriptomes

In [314]:
salmon_file = alignment_rate_dir + "salmon_denovo_alignrates"
salmon_data = pd.read_csv(salmon_file,sep=" ",header=None)
rate = salmon_data[[0,7]]
rate.columns = ['sample_info','perc_info']
rate['salmon_alignment_rate'] = rate.perc_info.str.split('%',n=1,expand=True)[0]
rate['sample'] = rate.sample_info.str.split("/",n=2,expand=True)[1]
df = rate[['sample','salmon_alignment_rate']]
df = df.drop_duplicates(subset='sample', keep='first', inplace=False)
df[['genus','species1','other1','other2']] = df['sample'].str.split('_',expand=True)
df["species"] = df["genus"] + "_" + df["species1"]
df = df[['sample','species','salmon_alignment_rate']]
df[['salmon_alignment_rate']] = df[['salmon_alignment_rate']].astype('float')

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [315]:
# get mean and sd of df_salmon groupby species
df_salmon_mean = df.groupby("species")["salmon_alignment_rate"].agg([np.mean, np.std])
df_salmon_mean.columns = ['mean_salmondenovo','std_salmondenovo']

In [316]:
df_salmon_denovo = pd.merge(df_salmon_mean,x_axis_jaccard,on = "species")

In [317]:
df_salmon_denovo

mean_salmondenovo  std_salmondenovo    rna.fa
species                                                          
A_xenica                    82.434400          2.554720  0.065359
F_catanatus                 81.595657          2.351519  0.109070
F_chrysotus                 81.800800          0.974929  0.066424
F_diaphanus                 80.270933          1.800028  0.114249
F_grandis                   74.955256          1.485886  0.121405
F_heteroclitusMDPL          75.755189          2.042814  0.221701
F_heteroclitusMDPP          74.023778          3.181816  0.201000
F_notatus                   81.458678          1.289989  0.073891
F_notti                     83.698650          2.822841  0.089826
F_olivaceous                82.859237          1.403167  0.078874
F_parvapinis                82.664937          2.018827  0.050145
F_rathbuni                  82.010856          1.775989  0.097641
F_sciadicus                 82.068325          2.058371  0.084369
F_similis                   75.993111          1.477629  0.065499
F_zebrinus                  81.141075          4.547584  0.082327
L_goodei                    77.046000          2.031459  0.054161
L_parva                     80.816611          1.434253  0.053764

# hisat2

In [318]:
# I know the files are labeled STAR, this was a typo in my script that named all my jobs with "STAR" prior to running on hpc

In [319]:
hisat2_file = alignment_rate_dir + "hisat2_genome_alignmentrate"
hisat2_data = pd.read_csv(hisat2_file,sep=" ",header=None)

In [320]:
rate = hisat2_data[[0]]
rate.columns = ['sample_info']
rate['hisat2_alignment_rate'] = rate.sample_info.str.split(':',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split(":",n=1,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [321]:
rate['sample'] = rate.samples.str.split("/",n=2,expand=True)[1]

In [322]:
rate[['star','genus','species1','condition','other1']] = rate['sample'].str.split('_',expand=True)

In [323]:
rate["species"] = rate["genus"] + "_" + rate["species1"]

In [324]:
rate['replicate'] = rate['genus'] + "_" + rate['species1'] + "_" + rate['condition'] + "_" + rate['other1'].str.split("-",expand=True)[0]

In [325]:
rate['alignment_rate'] = rate['hisat2_alignment_rate'].str.split("%",expand=True)[0]

In [326]:
rate = rate[['replicate','species','alignment_rate']]

In [327]:
rate[['alignment_rate']] = rate[['alignment_rate']].astype('float')

In [328]:
# get mean and sd of df_salmon groupby species
df_hisat2_mean = rate.groupby("species")["alignment_rate"].agg([np.mean, np.std])
df_hisat2_mean.columns = ['mean_hisat2','std_hisat2']

In [329]:
df_hisat2 = pd.merge(df_hisat2_mean,x_axis_jaccard,on = "species")

In [330]:
df_hisat2

mean_hisat2  std_hisat2    rna.fa
species                                              
A_xenica              31.945556    1.548710  0.065359
F_catanatus           48.815714    1.183960  0.109070
F_chrysotus           32.077500    1.028477  0.066424
F_diaphanus           49.873333    2.151712  0.114249
F_grandis             63.860000    1.497030  0.121405
F_heteroclitusMDPL    76.278889    2.315974  0.221701
F_heteroclitusMDPP    77.720000    2.681832  0.201000
F_notatus             36.821111    0.987743  0.073891
F_notti               32.405000    0.162635  0.089826
F_olivaceous          36.528750    0.816848  0.078874
F_parvapinis          26.710000    3.121263  0.050145
F_rathbuni            49.791111    1.988815  0.097641
F_sciadicus           35.585000    1.453513  0.084369
F_similis             36.833333    2.335974  0.065499
F_zebrinus            34.647500    0.983713  0.082327
L_goodei              29.051111    2.227467  0.054161
L_parva               27.895556    1.647818  0.053764

# bwa mem genome

In [331]:
bwagenome_file = alignment_rate_dir + "bwa_genome_alignmentrates"
bwagenome_data = pd.read_csv(bwagenome_file,sep=" ",header=None)

In [332]:
rate = bwagenome_data[[0,4]]
rate.columns = ['sample_info','perc_info']
rate['bwagenome_alignment_rate'] = rate.perc_info.str.split('(',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [333]:
rate['replicate'] = rate.samples.str.split(".",n=2,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [334]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] 

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [335]:
rate['bwa_mem_genome_alignment_rate'] = rate.bwagenome_alignment_rate.str.split("%",n=1,expand=True)[0].astype('float')

In [336]:
rate = rate[['replicate','species','bwa_mem_genome_alignment_rate']]

In [337]:
# get mean and sd of df_salmon groupby species
df_bwamemgenome_mean = rate.groupby("species")["bwa_mem_genome_alignment_rate"].agg([np.mean, np.std])
df_bwamemgenome_mean.columns = ['mean_bwamem_genome','std_bwamem_genome']

In [338]:
df_bwamemgenome = pd.merge(df_bwamemgenome_mean,x_axis_jaccard,on = "species")

In [339]:
df_bwamemgenome

mean_bwamem_genome  std_bwamem_genome    rna.fa
species                                                            
A_xenica                     91.551111           0.607195  0.065359
F_catanatus                  94.168333           0.841794  0.109070
F_chrysotus                  91.680000           0.478271  0.066424
F_diaphanus                  94.965000           0.374206  0.114249
F_grandis                    95.662500           0.428477  0.121405
F_heteroclitusMDPL           96.248889           0.788344  0.221701
F_heteroclitusMDPP           96.134444           0.781747  0.201000
F_notatus                    93.255000           0.605876  0.073891
F_notti                      91.920000           0.890955  0.089826
F_olivaceous                 92.625000           0.980379  0.078874
F_parvapinis                 87.833750           1.364844  0.050145
F_rathbuni                   95.573750           0.700692  0.097641
F_sciadicus                  92.660000           0.441588  0.084369
F_similis                    91.680000           0.479114  0.065499
F_zebrinus                   92.330000           0.809403  0.082327
L_goodei                     89.294444           0.713987  0.054161
L_parva                      89.652500           0.840693  0.053764

# bwa mem genome, clipping penalty -L 100 (length of read)

In [340]:
bwagenome_file = alignment_rate_dir + "bwa_genome_clipping_alignmentrates"
bwagenome_data = pd.read_csv(bwagenome_file,sep=" ",header=None)

In [341]:
rate = bwagenome_data[[0,4]]
rate.columns = ['sample_info','perc_info']
rate['bwagenome_clipping_alignment_rate'] = rate.perc_info.str.split('(',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [342]:
rate['replicate'] = rate.samples.str.split(".",n=2,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [343]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] 

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [344]:
rate['bwagenome_clipping_alignment_rate'] = rate.bwagenome_clipping_alignment_rate.str.split("%",n=1,expand=True)[0].astype('float')

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [345]:
rate = rate[['replicate','species','bwagenome_clipping_alignment_rate']]

In [346]:
# get mean and sd of df_salmon groupby species
df_bwamemgenome_mean = rate.groupby("species")['bwagenome_clipping_alignment_rate'].agg([np.mean, np.std])
df_bwamemgenome_mean.columns = ['mean_bwamem_genome_clipping','std_bwamem_genome_clipping']

In [347]:
df_bwamemgenome_clipping = pd.merge(df_bwamemgenome_mean,x_axis_jaccard,on = "species")

In [348]:
df_bwamemgenome_clipping

mean_bwamem_genome_clipping  std_bwamem_genome_clipping  \
species                                                                       
A_xenica                              91.158889                    0.623888   
F_catanatus                           93.947143                    0.922148   
F_chrysotus                           91.291250                    0.481944   
F_diaphanus                           94.626667                    0.393633   
F_grandis                             95.331111                    0.438077   
F_heteroclitusMDPL                    95.897778                    0.858673   
F_heteroclitusMDPP                    95.756667                    0.827964   
F_notatus                             92.826667                    0.624840   
F_notti                               91.535000                    0.940452   
F_olivaceous                          92.247500                    1.035605   
F_parvapinis                          87.402500                    1.385287   
F_rathbuni                            95.192222                    0.733634   
F_sciadicus                           92.287500                    0.458576   
F_similis                             91.287778                    0.502040   
F_zebrinus                            91.960000                    0.837536   
L_goodei                              88.881111                    0.721915   
L_parva                               89.182222                    0.820637   

                      rna.fa  
species                       
A_xenica            0.065359  
F_catanatus         0.109070  
F_chrysotus         0.066424  
F_diaphanus         0.114249  
F_grandis           0.121405  
F_heteroclitusMDPL  0.221701  
F_heteroclitusMDPP  0.201000  
F_notatus           0.073891  
F_notti             0.089826  
F_olivaceous        0.078874  
F_parvapinis        0.050145  
F_rathbuni          0.097641  
F_sciadicus         0.084369  
F_similis           0.065499  
F_zebrinus          0.082327  
L_goodei            0.054161  
L_parva             0.053764

# bwa mem genome, clipping/gap/mismatch penalties -L 100 -B 40 -O 60

In [349]:
bwagenome_file = alignment_rate_dir + "bwa_genome_mismatchgap_alignrates"
bwagenome_data = pd.read_csv(bwagenome_file,sep=" ",header=None)

In [350]:
rate = bwagenome_data[[0,4]]
rate.columns = ['sample_info','perc_info']
rate['bwagenome_clipping_alignment_rate'] = rate.perc_info.str.split('(',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [351]:
rate['replicate'] = rate.samples.str.split(".",n=2,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [352]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] 

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [353]:
rate['bwagenome_clipping_alignment_rate'] = rate.bwagenome_clipping_alignment_rate.str.split("%",n=1,expand=True)[0].astype('float')

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [354]:
rate = rate[['replicate','species','bwagenome_clipping_alignment_rate']]

In [355]:
# get mean and sd of df_salmon groupby species
df_bwamemgenome_mean = rate.groupby("species")['bwagenome_clipping_alignment_rate'].agg([np.mean, np.std])
df_bwamemgenome_mean.columns = ['mean_bwamem_genome_gapmismatch','std_bwamem_genome_gapmismatch']

In [356]:
df_bwamemgenome_gapmismatch = pd.merge(df_bwamemgenome_mean,x_axis_jaccard,on = "species")

In [357]:
df_bwamemgenome_gapmismatch

mean_bwamem_genome_gapmismatch  \
species                                              
A_xenica                                 82.977778   
F_catanatus                              90.097143   
F_chrysotus                              83.436250   
F_diaphanus                              90.650000   
F_grandis                                93.197500   
F_heteroclitusMDPL                       94.716667   
F_heteroclitusMDPP                       94.688889   
F_notatus                                86.233333   
F_notti                                  83.920000   
F_olivaceous                             85.782500   
F_parvapinis                             77.957500   
F_rathbuni                               91.464444   
F_sciadicus                              85.027500   
F_similis                                84.610000   
F_zebrinus                               84.650000   
L_goodei                                 79.633333   
L_parva                                  79.531111   

                    std_bwamem_genome_gapmismatch    rna.fa  
species                                                      
A_xenica                                 0.956279  0.065359  
F_catanatus                              1.120963  0.109070  
F_chrysotus                              0.809584  0.066424  
F_diaphanus                              0.601963  0.114249  
F_grandis                                0.541103  0.121405  
F_heteroclitusMDPL                       1.065094  0.221701  
F_heteroclitusMDPP                       1.117200  0.201000  
F_notatus                                0.804752  0.073891  
F_notti                                  0.989949  0.089826  
F_olivaceous                             0.978946  0.078874  
F_parvapinis                             2.094037  0.050145  
F_rathbuni                               1.142280  0.097641  
F_sciadicus                              0.766306  0.084369  
F_similis                                0.742849  0.065499  
F_zebrinus                               1.197637  0.082327  
L_goodei                                 1.373690  0.054161  
L_parva                                  1.284158  0.053764

# bwa mem txome

In [358]:
bwatranscriptome_file = alignment_rate_dir + "bwa_transcriptome_alignmentrates"
bwatranscriptome_data = pd.read_csv(bwatranscriptome_file,sep=" ",header=None)

In [359]:
rate = bwatranscriptome_data[[0,4]]
rate.columns = ['sample_info','perc_info']
rate['bwatranscriptome_alignment_rate'] = rate.perc_info.str.split('(',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [360]:
rate['replicate'] = rate.samples.str.split(".",n=2,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [361]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] 

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [362]:
rate['bwa_mem_transcriptome_alignment_rate'] = rate.bwatranscriptome_alignment_rate.str.split("%",n=1,expand=True)[0].astype('float')

In [363]:
rate = rate[['replicate','species','bwa_mem_transcriptome_alignment_rate']]

In [364]:
# get mean and sd of df_salmon groupby species
df_bwamemtxome_mean = rate.groupby("species")["bwa_mem_transcriptome_alignment_rate"].agg([np.mean, np.std])
df_bwamemtxome_mean.columns = ['mean_bwamem_txome','std_bwamem_txome']

In [365]:
df_bwamemtxome = pd.merge(df_bwamemtxome_mean,x_axis_jaccard,on = "species")

In [366]:
df_bwamemtxome

mean_bwamem_txome  std_bwamem_txome    rna.fa
species                                                          
A_xenica                    77.113333          1.542012  0.065359
F_catanatus                 74.998571          5.309317  0.109070
F_chrysotus                 76.215000          1.249846  0.066424
F_diaphanus                 77.216667          1.763073  0.114249
F_grandis                   68.971111         13.888326  0.121405
F_heteroclitusMDPL          75.183333          5.804731  0.221701
F_heteroclitusMDPP          77.158889          4.712485  0.201000
F_notatus                   75.898889          1.909021  0.073891
F_notti                     77.005000          1.520280  0.089826
F_olivaceous                73.971250          4.556256  0.078874
F_parvapinis                67.735000          4.518913  0.050145
F_rathbuni                  78.261111          1.738889  0.097641
F_sciadicus                 76.945000          3.064621  0.084369
F_similis                   66.975556          7.545426  0.065499
F_zebrinus                  72.620000          4.658905  0.082327
L_goodei                    68.804444          4.113417  0.054161
L_parva                     72.317778          4.830884  0.053764

# bwa mem txome, clipping penalty -L 100 (length of read)

In [367]:
bwatranscriptome_file = alignment_rate_dir + "bwa_txome_clipping_alignrates"
bwatranscriptome_data = pd.read_csv(bwatranscriptome_file,sep=" ",header=None)

In [368]:
rate = bwatranscriptome_data[[0,4]]
rate.columns = ['sample_info','perc_info']
rate['bwatxome_clipping_alignment_rate'] = rate.perc_info.str.split('(',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [369]:
rate['replicate'] = rate.samples.str.split(".",n=2,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [370]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] 

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [371]:
rate['bwatxome_clipping_alignment_rate'] = rate.bwatxome_clipping_alignment_rate.str.split("%",n=1,expand=True)[0].astype('float')

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [372]:
rate = rate[['replicate','species','bwatxome_clipping_alignment_rate']]

In [373]:
# get mean and sd of df_salmon groupby species
df_bwamemtxome_mean = rate.groupby("species")['bwatxome_clipping_alignment_rate'].agg([np.mean, np.std])
df_bwamemtxome_mean.columns = ['mean_bwamem_txome_clipping','std_bwamem_txome_clipping']

In [374]:
df_bwamemtxome_clipping = pd.merge(df_bwamemtxome_mean,x_axis_jaccard,on = "species")

In [375]:
df_bwamemtxome_clipping

mean_bwamem_txome_clipping  std_bwamem_txome_clipping  \
species                                                                     
A_xenica                             77.054444                   1.536499   
F_catanatus                          74.904286                   5.309086   
F_chrysotus                          76.153750                   1.246044   
F_diaphanus                          77.126667                   1.769731   
F_grandis                            68.871111                  13.890408   
F_heteroclitusMDPL                   75.073333                   5.817957   
F_heteroclitusMDPP                   77.031111                   4.722699   
F_notatus                            75.835556                   1.914080   
F_notti                              76.950000                   1.499066   
F_olivaceous                         73.902500                   4.565735   
F_parvapinis                         67.676250                   4.512915   
F_rathbuni                           78.185556                   1.743740   
F_sciadicus                          76.877500                   3.068565   
F_similis                            66.882222                   7.550437   
F_zebrinus                           72.550000                   4.678853   
L_goodei                             68.750000                   4.122678   
L_parva                              72.268889                   4.838229   

                      rna.fa  
species                       
A_xenica            0.065359  
F_catanatus         0.109070  
F_chrysotus         0.066424  
F_diaphanus         0.114249  
F_grandis           0.121405  
F_heteroclitusMDPL  0.221701  
F_heteroclitusMDPP  0.201000  
F_notatus           0.073891  
F_notti             0.089826  
F_olivaceous        0.078874  
F_parvapinis        0.050145  
F_rathbuni          0.097641  
F_sciadicus         0.084369  
F_similis           0.065499  
F_zebrinus          0.082327  
L_goodei            0.054161  
L_parva             0.053764

# bwa mem txome, clipping/gap/mismatch penalties -L 100 -B 40 -O 60

In [376]:
bwatranscriptome_file = alignment_rate_dir + "bwa_txome_mismatchgap_alignrates"
bwatranscriptome_data = pd.read_csv(bwatranscriptome_file,sep=" ",header=None)

In [377]:
rate = bwatranscriptome_data[[0,4]]
rate.columns = ['sample_info','perc_info']
rate['bwatxome_gapmismatch_alignment_rate'] = rate.perc_info.str.split('(',n=1,expand=True)[1]
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [378]:
rate['replicate'] = rate.samples.str.split(".",n=2,expand=True)[0]

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [379]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] 

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [380]:
rate['bwatxome_gapmismatch_alignment_rate'] = rate.bwatxome_gapmismatch_alignment_rate.str.split("%",n=1,expand=True)[0].astype('float')

/Users/johnsolk/anaconda3/envs/jupyterpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [381]:
rate = rate[['replicate','species','bwatxome_gapmismatch_alignment_rate']]

In [382]:
# get mean and sd of df_salmon groupby species
df_bwamemtxome_mean = rate.groupby("species")['bwatxome_gapmismatch_alignment_rate'].agg([np.mean, np.std])
df_bwamemtxome_mean.columns = ['mean_bwamem_txome_gapmismatch','std_bwamem_txome_gapmismatch']

In [383]:
df_bwamemtxome_gapmismatch = pd.merge(df_bwamemtxome_mean,x_axis_jaccard,on = "species")

In [384]:
df_bwamemtxome_gapmismatch

mean_bwamem_txome_gapmismatch  \
species                                             
A_xenica                                72.691111   
F_catanatus                             73.950000   
F_chrysotus                             72.077500   
F_diaphanus                             75.848333   
F_grandis                               67.486250   
F_heteroclitusMDPL                      75.572500   
F_heteroclitusMDPP                      76.523750   
F_notatus                               72.860000   
F_notti                                 72.740000   
F_olivaceous                            70.968750   
F_parvapinis                            62.428750   
F_rathbuni                              77.050000   
F_sciadicus                             73.222500   
F_similis                               64.115556   
F_zebrinus                              69.340000   
L_goodei                                63.604444   
L_parva                                 66.702222   

                    std_bwamem_txome_gapmismatch    rna.fa  
species                                                     
A_xenica                                1.595998  0.065359  
F_catanatus                             5.133806  0.109070  
F_chrysotus                             1.154194  0.066424  
F_diaphanus                             1.561735  0.114249  
F_grandis                              13.950704  0.121405  
F_heteroclitusMDPL                      5.274320  0.221701  
F_heteroclitusMDPP                      5.120058  0.201000  
F_notatus                               1.894353  0.073891  
F_notti                                 1.173797  0.089826  
F_olivaceous                            4.440455  0.078874  
F_parvapinis                            4.056696  0.050145  
F_rathbuni                              1.944158  0.097641  
F_sciadicus                             2.551488  0.084369  
F_similis                               6.985848  0.065499  
F_zebrinus                              4.353420  0.082327  
L_goodei                                3.482144  0.054161  
L_parva                                 4.423776  0.053764

# STAR

In [385]:
star_file = alignment_rate_dir + "STAR_genome_alignmentrates"
star_data = pd.read_csv(star_file,sep="\t",header=None)

In [386]:
rate = star_data
rate.columns = ['sample_info','perc_info']
rate['STAR_genome_alignment_rate'] = rate.perc_info.str.split('%',n=1,expand=True)[0].astype('float')

In [387]:
rate['samples'] = rate.sample_info.str.split("/",n=2,expand=True)[1]

In [388]:
rate['species'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1]

In [389]:
rate['replicate_info'] = rate.samples.str.split("_",n=4,expand=True)[3]

In [390]:
rate['replicate'] = rate.samples.str.split("_",n=4,expand=True)[0] + "_" + rate.samples.str.split("_",n=4,expand=True)[1] + "_" + rate.samples.str.split("_",n=4,expand=True)[2] + "_" + rate.replicate_info.str.split("Log",n=1,expand=True)[0]


In [391]:
rate = rate[['replicate','species','STAR_genome_alignment_rate']]

In [392]:
# get mean and sd of df_salmon groupby species
df_star_mean = rate.groupby("species")["STAR_genome_alignment_rate"].agg([np.mean, np.std])
df_star_mean.columns = ['mean_star','std_star']

In [393]:
df_star = pd.merge(df_star_mean,x_axis_jaccard,on = "species")

In [394]:
df_star

mean_star  std_star    rna.fa
species                                          
A_xenica            55.595556  2.006116  0.065359
F_catanatus         69.924286  2.742528  0.109070
F_chrysotus         56.520000  1.459011  0.066424
F_diaphanus         70.513333  1.951591  0.114249
F_grandis           75.825556  3.200383  0.121405
F_heteroclitusMDPL  81.781111  3.009009  0.221701
F_heteroclitusMDPP  82.238889  3.484194  0.201000
F_notatus           60.947778  1.573243  0.073891
F_notti             57.005000  1.294005  0.089826
F_olivaceous        60.043750  1.869048  0.078874
F_parvapinis        46.515000  2.944287  0.050145
F_rathbuni          72.427778  2.236575  0.097641
F_sciadicus         58.667500  1.812078  0.084369
F_similis           56.155556  1.478691  0.065499
F_zebrinus          57.905000  2.036165  0.082327
L_goodei            49.967778  1.637580  0.054161
L_parva             50.121111  2.281307  0.053764

# 6 reference alignment methods + salmon de novo together

In [395]:
alldf = pd.merge(df_star,df_bwamemtxome,on="species")
alldf = alldf.drop(columns=['rna.fa_x', 'rna.fa_y'])
alldf = pd.merge(alldf,df_bwamemgenome,on="species")
alldf = pd.merge(alldf,df_hisat2,on="species")
alldf = alldf.drop(columns=['rna.fa_x', 'rna.fa_y'])
alldf = pd.merge(alldf,df_salmon,on="species")
alldf = alldf.rename(index=str, columns={"rna.fa": "Fhet_jaccard"})
alldf = pd.merge(alldf,df_salmon_denovo,on="species")
alldf = alldf.drop(columns=['rna.fa'])
alldf = pd.merge(alldf,df_bwamemgenome_clipping,on="species")
alldf = alldf.drop(columns=['rna.fa'])
alldf = pd.merge(alldf,df_bwamemtxome_clipping,on="species")
alldf = alldf.drop(columns=['rna.fa'])
alldf = pd.merge(alldf,df_bwamemgenome_gapmismatch,on="species")
alldf = alldf.drop(columns=['rna.fa'])
alldf = pd.merge(alldf,df_bwamemtxome_gapmismatch,on="species")
alldf = alldf.drop(columns=['rna.fa'])

In [396]:
alldf[['Fhet_jaccard']] = alldf[['Fhet_jaccard']].astype('float')

In [397]:
alldf = alldf.sort_values(by=['Fhet_jaccard'],ascending=True)

In [398]:
# This is the dataframe to be plotted.
alldf

mean_star  std_star  mean_bwamem_txome  std_bwamem_txome  \
species                                                                        
F_parvapinis        46.515000  2.944287          67.735000          4.518913   
L_parva             50.121111  2.281307          72.317778          4.830884   
L_goodei            49.967778  1.637580          68.804444          4.113417   
A_xenica            55.595556  2.006116          77.113333          1.542012   
F_similis           56.155556  1.478691          66.975556          7.545426   
F_chrysotus         56.520000  1.459011          76.215000          1.249846   
F_notatus           60.947778  1.573243          75.898889          1.909021   
F_olivaceous        60.043750  1.869048          73.971250          4.556256   
F_zebrinus          57.905000  2.036165          72.620000          4.658905   
F_sciadicus         58.667500  1.812078          76.945000          3.064621   
F_notti             57.005000  1.294005          77.005000          1.520280   
F_rathbuni          72.427778  2.236575          78.261111          1.738889   
F_catanatus         69.924286  2.742528          74.998571          5.309317   
F_diaphanus         70.513333  1.951591          77.216667          1.763073   
F_grandis           75.825556  3.200383          68.971111         13.888326   
F_heteroclitusMDPP  82.238889  3.484194          77.158889          4.712485   
F_heteroclitusMDPL  81.781111  3.009009          75.183333          5.804731   

                    mean_bwamem_genome  std_bwamem_genome  mean_hisat2  \
species                                                                  
F_parvapinis                 87.833750           1.364844    26.710000   
L_parva                      89.652500           0.840693    27.895556   
L_goodei                     89.294444           0.713987    29.051111   
A_xenica                     91.551111           0.607195    31.945556   
F_similis                    91.680000           0.479114    36.833333   
F_chrysotus                  91.680000           0.478271    32.077500   
F_notatus                    93.255000           0.605876    36.821111   
F_olivaceous                 92.625000           0.980379    36.528750   
F_zebrinus                   92.330000           0.809403    34.647500   
F_sciadicus                  92.660000           0.441588    35.585000   
F_notti                      91.920000           0.890955    32.405000   
F_rathbuni                   95.573750           0.700692    49.791111   
F_catanatus                  94.168333           0.841794    48.815714   
F_diaphanus                  94.965000           0.374206    49.873333   
F_grandis                    95.662500           0.428477    63.860000   
F_heteroclitusMDPP           96.134444           0.781747    77.720000   
F_heteroclitusMDPL           96.248889           0.788344    76.278889   

                    std_hisat2  mean_salmonFhet  std_salmonFhet  ...  \
species                                                          ...   
F_parvapinis          3.121263        44.248000        2.866265  ...   
L_parva               1.647818        49.292456        4.072149  ...   
L_goodei              2.227467        46.687689        2.752060  ...   
A_xenica              1.548710        55.475511        1.766508  ...   
F_similis             2.335974        48.177767        4.917644  ...   
F_chrysotus           1.028477        54.770188        1.296955  ...   
F_notatus             0.987743        56.591278        2.152217  ...   
F_olivaceous          0.816848        55.028838        3.442983  ...   
F_zebrinus            0.983713        52.972925        3.926850  ...   
F_sciadicus           1.453513        56.682275        1.876854  ...   
F_notti               0.162635        55.536150        0.617092  ...   
F_rathbuni            1.988815        61.714956        1.894362  ...   
F_catanatus           1.183960        59.184429        4.343153  ...   
F_diaphanus 

In [400]:
def plotall():
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.set_facecolor('white')
    plt.errorbar(alldf['Fhet_jaccard'],alldf['mean_star'], alldf['std_star'], lw=2, marker='o', color = 'red')
    #ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_bwamem_txome'], alldf['std_bwamem_txome'],lw=2, marker='o', color = 'blue')
    #ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_bwamem_genome'], alldf['std_bwamem_genome'],lw=2, marker='o', color = 'green')
    ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_hisat2'], alldf['std_hisat2'],lw=2, marker='o', color = 'green')
    ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_salmonFhet'], alldf['std_salmonFhet'],lw=2, marker='o', color = 'purple')
    ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_salmondenovo'], alldf['std_salmondenovo'],lw=2, marker='o', linestyle='--', color = 'black')
    #ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_bwamem_genome_clipping'],alldf['std_bwamem_genome_clipping'],lw=2, marker='o', color = 'orange')
    #ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_bwamem_txome_clipping'],alldf['std_bwamem_txome_clipping'],lw=2, marker='o', color = 'cyan')
    ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_bwamem_genome_gapmismatch'],alldf['std_bwamem_genome_gapmismatch'],lw=2, marker='o', color = 'orange')
    ax.errorbar(alldf['Fhet_jaccard'],alldf['mean_bwamem_txome_gapmismatch'],alldf['std_bwamem_txome_gapmismatch'],lw=2, marker='o', color = 'cyan')
    ax.legend(loc='lower left', frameon=False,fontsize=16)
    fig = ax.get_figure()
    ax.invert_xaxis()
    ax.set_xscale("log")
    plt.yticks(fontsize=14)
    plt.ylim((25,100))
    ax.set_yticks(np.arange(25, 100, 10),minor=True,)
    ax.yaxis.set_tick_params(length=10)
    ax.yaxis.set_tick_params(which="minor",length=5)
    plt.xticks(alldf['Fhet_jaccard'],alldf.index, rotation=90,fontsize=14)
    ax.set_ylabel('Alignment %', fontsize=18)
    ax.set_xlabel('Species (jaccard distance of de novo assembly from NCBI F. heteroclitus reference transcriptome)', fontsize=18)
    fig.show()

plotall()
